## Evaluate Smolagent framework

### Ability
- This agent will have access to web search via duckduckgo
### Purpose
- The task of agent is to provide you list of books for tech industry professionals on a given topic

### Model used
- Qwen/Qwen2.5-Coder-32B-Instruct (default)
- Qwen/Qwen3-Next-80B-A3B-Instruct

In [ ]:
import os

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

## first we would use codeAgent
- codeAgent means the agent will try to write code and use the same to arrive at result

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = CodeAgent(
	tools=[DuckDuckGoSearchTool()],
	model=InferenceClientModel(
		api_key=os.getenv("HF_WRITE_TOKEN"),
		provider="together",
		model_id="Qwen/Qwen3-Next-80B-A3B-Instruct",
	),
)
agent.run(
	"Search for the best book recommendation for getting good at programming in rust for programmer who know python"
)

In [ ]:
agent.run(
	"I want to scale my production rag application, what is the best resource to learn and implement for my company."
)

In [ ]:
agent_reg = CodeAgent(
	tools=[DuckDuckGoSearchTool()],
	model=InferenceClientModel(
		api_key=os.getenv("HF_WRITE_TOKEN"), provider="together"
	),
)

In [ ]:
agent_reg.run(
	"Search for the best book recommendation for getting good at programming in rust for programmer who know python"
)

In [ ]:
agent_reg.run(
	"I want to scale my production rag application, what is the best resource to learn and implement for my company."
)

## Let's try the same with toolagent now
- it will interact with `json` for input output and communication

In [ ]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, InferenceClientModel

In [ ]:
tool_agent_reg = ToolCallingAgent(
	tools=[DuckDuckGoSearchTool()],
	model=InferenceClientModel(
		provider="together", api_key=os.getenv("HF_WRITE_TOKEN")
	),
)

In [ ]:
tool_agent = ToolCallingAgent(
	tools=[DuckDuckGoSearchTool()],
	model=InferenceClientModel(
		provider="together",
		model_id="Qwen/Qwen3-Next-80B-A3B-Instruct",
		api_key=os.getenv("HF_WRITE_TOKEN"),
	),
)

In [ ]:
tool_agent_reg.run(
	"I want to scale my production rag application, what is the best resource to learn and implement for my company."
)

In [ ]:
tool_agent_reg.run(
	"Search for the best book recommendation for getting good at programming in rust for programmer who know python"
)

|          Feature          | ToolCallingAgent              |              CodeAgent              |
|:-------------------------:|-------------------------------|:-----------------------------------:|
| **Complexity**            | Lower complexity              | Higher complexity                   |
| **Flexibility**           | Structured and predictable    | Highly flexible and dynamic         |
| **Arbitrary Code**        | No                            | Yes                                 |
| **Typical tasks**         | Simple, structured tasks      | Complex, unstructured tasks         |
| **Error risk**            | Lower (predictable execution) | Higher (can have runtime errors)    |
| **3rd party packages**    | No                            | Can integrate to improve output     |
| **Security implications** | Safer (API calls only)        | Needs caution(executes python code) |

## Let's integrate LangFuse and opentelemetry to see behind the scene in agent execution

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
LANGFUSE_PUBLIC_KEY = os.getenv("PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("SECRET_KEY")

In [ ]:
# OTEL_EXPORTER_OTLP_ENDPOINT = "https://cloud.langfuse.com/api/public/otel"  # EU Region
# OTEL_EXPORTER_OTLP_ENDPOINT = "https://us.cloud.langfuse.com/api/public/otel"

In [ ]:
from langfuse import Langfuse

langfuse = Langfuse(
	secret_key=os.getenv("SECRET_KEY"),
	public_key=os.getenv("PUBLIC_KEY"),
	host=os.getenv("LANGFUSE_HOST"),
)

In [ ]:
from langfuse import get_client

langfuse = get_client()

# Verify connection, do not use in production as this is a synchronous call
if langfuse.auth_check():
	print("Langfuse client is authenticated and ready!")
else:
	print("Authentication failed. Please check your credentials and host.")

In [ ]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [ ]:
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

In [ ]:
SmolagentsInstrumentor().instrument(trace_provider=trace_provider)

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

agent_langfuse = CodeAgent(
	tools=[DuckDuckGoSearchTool()],
	model=InferenceClientModel(
		provider="together", api_key=os.getenv("HF_WRITE_TOKEN")
	),
)

In [ ]:
agent_langfuse.run(
	"Search the best book for implementing agents for production using rust language."
)

In [ ]:
from smolagents import ToolCallingAgent, InferenceClientModel

tool_agent_langfuse = ToolCallingAgent(
	tools=[DuckDuckGoSearchTool()],
	model=InferenceClientModel(
		provider="together", api_key=os.getenv("HF_WRITE_TOKEN")
	),
)

In [ ]:
tool_agent_langfuse.run(
	"Search the best book for implementing agents for production using rust language."
)